## Blohova sfera

**Blohova sfera** je geometrijski prikaz jednog kubita, gde svako čisto stanje odgovara tački na površini sfere.  Severni pol predstavlja stanje $|0\rangle$, južni pol stanje $|1\rangle$, dok sve druge tačke predstavljaju superpozicije ovih baznih stanja.  

**Kvantne operacije** deluju kao rotacije vektora stanja kubita oko osa sfere (X, Y, Z).  Na primer, operator $R_y(\theta)$ rotira vektor stanja za ugao $\theta$ oko Y-ose, dok operator $S$ vrši rotaciju od $\pi/2$ oko Z-ose.  

![Bloch-Sphere](images/Bloch-Sphere.png)

- Ova laboratorijska vežba prikazuje kako se vektor stanja menja na Blohovoj sferi nakon primene različitih jednokubitnih operacija.  
- Možete uneti liste operacija (npr. `["h"]`, `["ry(0.7)"]`, `["h","rz(pi/4)","ry(1.2)"]`) i posmatrati nastale rotacije.  
- Program prikazuje dve Blohove sfere uporedo, što omogućava poređenje dejstva različitih nizova operacija.

---

### Zadatak

1. Ova  vežba **ne zahteva** pisanje ili analiziranje koda.
2. Zadatak je da eksperimentišete sa različitim sekvencama jednokubitnih operacija i posmatrate kako se vektor stanja pomera na Blohovoj sferi.  
3. Pokrenite dati program da biste generisali dve Blohove sfere na osnovu `gates_set_1` i `gates_set_2`.  
4. Izmenite vrednosti `gates_set_1` i `gates_set_2` (koristite operacije kao što su `h`, `x`, `y`, `z`, `rx(θ)`, `ry(θ)`, `rz(θ)`, `p(φ)`, ili `""`/`"i"` za identitet) i ponovo pokrenite program.  
5. Posmatrajte kako se vektor stanja pomera na svakoj Blohovoj sferi i objasnite dejstvo svake operacije (npr. `ry(θ)` rotira u realnoj \(x\)–\(z\) ravni; `rz(θ)` menja fazu; `x` obrće \(z\)-koordinatu).  
6. Uporedite dva rezultata i opišite razlike između leve i desne sfere, kao i kako različite sekvence operacija (prikazane u naslovima) dovode do tih razlika.


In [ ]:
import re
import math
import textwrap
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization.bloch import Bloch

# ---------- parsing + circuit ----------
def _parse_gate_string(s):
    s = s.strip().lower()
    if s == "" or s == "i":
        return "i", []
    if "(" not in s and ")" not in s:
        return s, []
    m = re.match(r'^([a-z][a-z0-9_]*)\s*\((.*)\)$', s)
    if not m:
        raise ValueError(f"Could not parse gate string: {s}")
    name = m.group(1)
    args_str = m.group(2).strip()
    safe_globals = {"__builtins__": None}
    safe_locals = {"pi": math.pi, "tau": math.tau}
    params = [eval(arg.strip(), safe_globals, safe_locals)
              for arg in args_str.split(",") if arg.strip()]
    return name, params

def generate_qc(gates, num_qubits=1, default_qubit=0):
    """
    Apply gates given as strings (e.g. 'h', 'ry(1.2)', 'rz(pi/4)') to default_qubit
    and return the final Statevector. 'i' or '' = identity (do nothing).
    """
    qc = QuantumCircuit(num_qubits)
    allowed = {
        "i", "h", "x", "y", "z", "s", "sdg", "t", "tdg",
        "rx", "ry", "rz", "p"
    }
    for g in gates:
        if not isinstance(g, str):
            raise TypeError(f"Gate must be a string, got: {type(g)}")
        name, params = _parse_gate_string(g)
        if name == "i":
            continue
        if name not in allowed or not hasattr(qc, name):
            raise ValueError(f"Unsupported gate: {name}")
        getattr(qc, name)(*params, default_qubit)
    return Statevector.from_instruction(qc)


def bloch_vector_from_state(state):
    """Return Bloch vector (x,y,z) for a single-qubit Statevector."""
    a, b = state.data[0], state.data[1]
    x = 2 * np.real(np.conj(a) * b)
    y = 2 * np.imag(np.conj(a) * b)
    z = np.abs(a)**2 - np.abs(b)**2
    return [x, y, z]

def title_from_gates(gates, width=48):
    """Create a wrapped title from a list of gate strings."""
    txt = "Applied gates: " + ", ".join(gates) if gates else "Applied gates: (none)"
    return "\n".join(textwrap.wrap(txt, width=width))

def plot_two_bloch(gates1, gates2, default_qubit=0):
    """Plot Bloch spheres for two different gate sequences."""
    state1 = generate_qc(gates1, num_qubits=1, default_qubit=default_qubit)
    state2 = generate_qc(gates2, num_qubits=1, default_qubit=default_qubit)

    v1 = bloch_vector_from_state(state1)
    v2 = bloch_vector_from_state(state2)

    fig = plt.figure(figsize=(11, 4))
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')

    b1 = Bloch(fig=fig, axes=ax1); b1.add_vectors(v1); b1.render(title=title_from_gates(gates1))
    b2 = Bloch(fig=fig, axes=ax2); b2.add_vectors(v2); b2.render(title=title_from_gates(gates2))

    plt.tight_layout()
    plt.show()

# ------------------------------------------------
#                 main program
# ------------------------------------------------

gates_set_1 = ["I"]   # user edits here
gates_set_2 = ["ry(1.2)","s"]

plot_two_bloch(gates_set_1, gates_set_2)
